In [ ]:
# ===============================
# 1. Imports
# ===============================

import numpy as np
import joblib

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Pauli


# ===============================
# 2. Quantum Feature Map
# ===============================

def quantum_feature_map(x):
    """
    Fixed quantum circuit used only for feature preparation
    """
    n_qubits = len(x)
    qc = QuantumCircuit(n_qubits)

    # Angle encoding
    for i in range(n_qubits):
        qc.ry(x[i], i)
        qc.rz(x[i], i)

    # Linear entanglement
    for i in range(n_qubits - 1):
        qc.cx(i, i + 1)

    return qc


def extract_quantum_features(x):
    """
    Extract expectation-value-based quantum features
    """
    qc = quantum_feature_map(x)
    state = Statevector.from_instruction(qc)

    features = []

    # ⟨Z_i⟩
    for i in range(len(x)):
        pauli = ['I'] * len(x)
        pauli[i] = 'Z'
        features.append(
            np.real(state.expectation_value(Pauli(''.join(pauli))))
        )

    # ⟨Z_i Z_{i+1}⟩
    for i in range(len(x) - 1):
        pauli = ['I'] * len(x)
        pauli[i] = 'Z'
        pauli[i + 1] = 'Z'
        features.append(
            np.real(state.expectation_value(Pauli(''.join(pauli))))
        )

    return np.array(features)


# ===============================
# 3. Training Pipeline
# ===============================

def train_pipeline(X, y, model_dir="models"):
    """
    Train full hybrid QML pipeline and save artifacts
    """

    # ---- PCA ----
    pca = PCA(n_components=8)
    X_pca = pca.fit_transform(X)

    # ---- Scaling to [0, π] ----
    scaler = MinMaxScaler(feature_range=(0, np.pi))
    X_scaled = scaler.fit_transform(X_pca)

    # ---- Quantum feature extraction ----
    Q_features = np.array([
        extract_quantum_features(x) for x in X_scaled
    ])

    # ---- Train / test split ----
    X_train, X_test, y_train, y_test = train_test_split(
        Q_features, y, test_size=0.2, random_state=42
    )

    # ---- Classical regression ----
    regressor = RandomForestRegressor(
        n_estimators=200,
        max_depth=10,
        random_state=42
    )
    regressor.fit(X_train, y_train)

    # ---- Evaluation ----
    y_pred = regressor.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"[TRAINING] Test MSE: {mse:.4f}")

    # ---- Save artifacts ----
    joblib.dump(pca, f"{model_dir}/pca.pkl")
    joblib.dump(scaler, f"{model_dir}/scaler.pkl")
    joblib.dump(regressor, f"{model_dir}/regressor.pkl")

    print("[TRAINING] Model artifacts saved")

    return mse


# ===============================
# 4. Inference Pipeline
# ===============================

def load_pipeline(model_dir="models"):
    """
    Load trained pipeline components
    """
    pca = joblib.load(f"{model_dir}/pca.pkl")
    scaler = joblib.load(f"{model_dir}/scaler.pkl")
    regressor = joblib.load(f"{model_dir}/regressor.pkl")

    return pca, scaler, regressor


def predict(X_new, pca, scaler, regressor):
    """
    End-to-end inference on new data
    """

    # Same preprocessing as training
    X_pca = pca.transform(X_new)
    X_scaled = scaler.transform(X_pca)

    # Quantum feature preparation
    Q_features = np.array([
        extract_quantum_features(x) for x in X_scaled
    ])

    # Prediction
    return regressor.predict(Q_features)


# ===============================
# 5. Main (Example Usage)
# ===============================

if __name__ == "__main__":

    # ---- Dummy dataset ----
    N = 10000
    X = np.random.rand(N, 40)
    y = np.sum(X[:, :5], axis=1) + 0.1 * np.random.randn(N)

    # ---- Train ----
    train_pipeline(X, y)

    # ---- Load pipeline ----
    pca, scaler, regressor = load_pipeline()

    # ---- Inference ----
    X_new = np.random.rand(5, 40)
    predictions = predict(X_new, pca, scaler, regressor)

    print("[INFERENCE] Predictions:", predictions)
